In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
data = pd.read_csv('data/BBAG-cross.csv')

In [4]:
data = data[data.country != 'Slovakia'].reset_index(drop =  True)

In [5]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


In [6]:
data.columns

Index(['Unnamed: 0', 'record_id', 'country', 'year_acq_x', 'Sex_1F_2M',
       'Education', 'Barthel', 'Diabetes_1Y_0N', 'Hypertension_1Y_0N',
       'Heart_Disease_1Y_0N', 'Physical_activity_1Y_0N', 'Cognition',
       'Well_being_domain', 'Sleep_problems_1Y_0N', 'Audition_problems',
       'Vision_problems', 'Age', 'Mono', 'One', 'Two', 'Three', 'Total',
       'total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
       'GDP_per_capita', 'gender_equal_l', 'Polution_conc_inv', 'Eq',
       'free_parties_l', 'inclu_suff_est', 'cred_elect_est', 'local_dem_est',
       'Migration', 'number_leng_inst', 'number_stable_inst', 'distance',
       'Proficiency', 'Educ_doct', 'Educ_upper_sec', 'Educ_master',
       'Educ_bach', 'GAP_corrected', 'GAP_bin', 'y_pred_corrected'],
      dtype='object')

## Odd ratios with macrosocials (new vars)

### Without Iteration

In [7]:
results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
        
        y_ols = c_results_merge_df_all['GAP_bin']
        X_ols = c_results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [8]:
for covar in covar_list:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-removing-slovakia_new-covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfOR)

GDP_per_capita


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,1.593654,1.825603,1.705690,15.404104,1.536029e-53,Mono,GDP_per_capita
1,0.518442,0.609849,0.562291,-13.898152,6.499814e-44,One,GDP_per_capita
2,0.585993,0.682556,0.632434,-11.774401,5.289155e-32,Two,GDP_per_capita
3,1.025799,1.223144,1.120134,2.527438,1.148980e-02,Three,GDP_per_capita
4,0.547905,0.627736,0.586464,-15.379182,2.257806e-53,Total,GDP_per_capita


GDP


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.058225,2.335952,2.192696,24.314864,1.365052e-130,Mono,GDP
6,0.695686,0.816104,0.753494,-6.949705,3.660498e-12,One,GDP
7,0.474929,0.552415,0.512209,-17.352100,1.901630e-67,Two,GDP
8,0.592966,0.692336,0.640727,-11.262546,2.008711e-29,Three,GDP
9,0.428002,0.485801,0.455987,-24.301625,1.884289e-130,Total,GDP


Educ_doct


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
10,1.725634,2.101130,1.904149,12.822859,1.221132e-37,Mono,Educ_doct
11,0.669182,0.822077,0.741700,-5.692139,1.254574e-08,One,Educ_doct
12,0.549977,0.673534,0.608629,-9.604307,7.667168e-22,Two,Educ_doct
13,0.734711,0.953600,0.837031,-2.674171,7.491432e-03,Three,Educ_doct
14,0.475704,0.579196,0.524905,-12.835221,1.041053e-37,Total,Educ_doct


Educ_bach


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
15,1.660865,2.002528,1.823713,12.590836,2.371475e-36,Mono,Educ_bach
16,0.598770,0.725107,0.658917,-8.541657,1.323108e-17,One,Educ_bach
17,0.682262,0.862716,0.767202,-4.426639,9.571271e-06,Two,Educ_bach
18,0.772049,0.929708,0.847219,-3.497451,4.697274e-04,Three,Educ_bach
19,0.499543,0.602288,0.548515,-12.585796,2.527827e-36,Total,Educ_bach


### 1000-Iteration

In [9]:

results_merge_df_all = data.copy()

covar_list = (
                'GDP_per_capita', 'GDP', 'Educ_doct', 'Educ_bach'
)


n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            c_results_merge_df_all = results_merge_df_all[[i, covar, 'GAP_bin']].dropna().reset_index(drop = True)
            
            y_ols = c_results_merge_df_all['GAP_bin']
            X_ols = c_results_merge_df_all[[i, covar]]

            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)



In [10]:


features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100), 
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [11]:
for covar in covar_list:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-removing-slovakia_new-covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfOR)

GDP_per_capita


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,GDP_per_capita,Mono,0.334782,0.382169,0.992084,1.708373,1.705857,1.710890,1.602197,1.819968,10.0
1,GDP_per_capita,One,0.382948,0.427415,0.990440,0.560930,0.559306,0.562553,0.518958,0.606624,10.0
2,GDP_per_capita,Two,0.113601,0.289453,0.989655,0.644646,0.643126,0.646166,0.599738,0.692172,10.0
3,GDP_per_capita,Three,0.157663,0.316281,0.989483,1.100255,1.098025,1.102486,1.012933,1.193216,10.0
4,GDP_per_capita,Total,0.330972,0.406004,0.992282,0.585470,0.584606,0.586335,0.549538,0.624308,10.0


GDP


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
5,GDP,Mono,0.252715,0.366058,0.991843,2.187131,2.184496,2.189765,2.062294,2.319312,10.0
6,GDP,One,0.146646,0.285495,0.972517,0.744695,0.742462,0.746929,0.690519,0.804596,10.0
7,GDP,Two,0.145421,0.321707,0.987474,0.524917,0.523691,0.526142,0.489139,0.563326,10.0
8,GDP,Three,0.234036,0.293858,0.981082,0.637490,0.636053,0.638927,0.592014,0.685901,10.0
9,GDP,Total,0.250820,0.421725,0.991387,0.457098,0.456547,0.457650,0.431023,0.484788,10.0


Educ_doct


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
10,Educ_doct,Mono,0.095305,0.661255,0.999979,1.891892,1.883911,1.899873,1.719745,2.077708,10.0
11,Educ_doct,One,0.110688,0.557477,0.999875,0.738723,0.735242,0.742205,0.668627,0.816225,10.0
12,Educ_doct,Two,0.198750,0.714116,0.999954,0.613226,0.610846,0.615606,0.556677,0.675265,10.0
13,Educ_doct,Three,0.072121,0.591721,0.999904,0.844758,0.839860,0.849656,0.741790,0.954762,10.0
14,Educ_doct,Total,0.094801,0.645199,0.999976,0.528327,0.526100,0.530554,0.481088,0.581189,10.0


Educ_bach


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
15,Educ_bach,Mono,0.149761,0.564045,0.999705,1.801264,1.793612,1.808915,1.647335,1.974012,10.0
16,Educ_bach,One,0.290119,0.491884,0.999596,0.646174,0.644213,0.648136,0.589571,0.706674,10.0
17,Educ_bach,Two,0.134159,0.495611,0.999507,0.785417,0.782331,0.788502,0.701973,0.876253,10.0
18,Educ_bach,Three,0.104876,0.387671,0.999610,0.864210,0.861355,0.867064,0.790450,0.945178,10.0
19,Educ_bach,Total,0.156140,0.547971,0.999739,0.555376,0.553022,0.557730,0.506775,0.607257,10.0
